In [12]:
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# 1. 브라우저 설정 (옵션 설정)
options = webdriver.ChromeOptions()
# options.add_argument('headless') # 브라우저 창 안 뜨게 하려면 주석 해제
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ==========================================
# STEP 1: Top 100 리스트에서 URL 수집하기
# ==========================================
driver.get("https://www.card-gorilla.com/chart/top100")
time.sleep(3) # 페이지 로딩 대기

card_links = []

try:
    # 1. 먼저 카드 이름이 포함된 'div.name_area'를 모두 찾습니다.
    # (가져오신 태그 구조상 가장 확실한 특징입니다)
    name_areas = driver.find_elements(By.CSS_SELECTOR, "div.name_area")
    
    for area in name_areas:
        # 2. 각 name_area의 바로 부모(parent) 태그인 <a>를 찾습니다.
        # XPath "./.."는 '현재 요소의 부모'를 의미합니다.
        parent_a_tag = area.find_element(By.XPATH, "./..")
        
        # 3. href 속성(링크) 추출
        link = parent_a_tag.get_attribute("href")
        
        # Selenium은 href를 가져올 때 자동으로 앞에 'https://...'를 붙여주지만,
        # 만약 '/card/detail/...'만 나온다면 앞에 도메인을 붙여야 합니다.
        # 여기서는 중복 체크 후 리스트에 담습니다.
        if link and link not in card_links:
            card_links.append(link)

    print(f"총 {len(card_links)}개의 카드 링크를 수집했습니다.")
    print(f"수집된 링크 예시: {card_links[0]}")

except Exception as e:
    print(f"에러 발생: {e}")

# ==========================================
# STEP 2: 상세 페이지 순회하며 정보 수집
# ==========================================
# 수집된 데이터를 담을 리스트
final_data = []

print("상세 페이지 크롤링을 시작합니다...")

for index, link in enumerate(card_links):
    print(f"[{index+1}/{len(card_links)}] 이동 중: {link}")
    
    try:
        driver.get(link)
        time.sleep(2)  # 페이지 로딩 대기
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 1. 기본 정보 추출
        # ---------------------------------------------------------
        # 카드명 (<strong class="card">)
        card_name_tag = soup.select_one("strong.card")
        card_name = card_name_tag.text.strip() if card_name_tag else "N/A"
        
        # 카드사 (<p class="brand">)
        corp_name_tag = soup.select_one("p.brand")
        corp_name = corp_name_tag.text.strip() if corp_name_tag else "N/A"
        
        # 연회비 (<dd class="in_out">)
        fee_tag = soup.select_one("dd.in_out")
        annual_fee = fee_tag.text.strip().replace('\n', ' ') if fee_tag else "N/A"
        
        # 2. 전월실적 추출 (특정 텍스트로 찾기)
        # ---------------------------------------------------------
        # <dt>전월실적</dt> 텍스트를 가진 태그를 찾고, 그 바로 다음 형제인 <dd>를 가져옴
        record_tag = soup.find("dt", string="전월실적")
        if record_tag:
            # find_next_sibling으로 바로 옆의 <dd> 태그 내용 가져오기
            performance = record_tag.find_next_sibling("dd").text.strip()
        else:
            performance = "N/A"

        # 3. 주요 혜택 리스트 추출
        # ---------------------------------------------------------
        # <div class="lst bene_area"> 안의 모든 <dl> 태그 찾기
        benefits_list = []
        benefit_area = soup.select("div.bene_area dl")
        
        for dl in benefit_area:
            try:
                # 혜택 카테고리 (예: 공과금, 편의점) -> <p class="txt1">
                category = dl.select_one("p.txt1").text.strip()
                
                # 혜택 상세 (예: 10% 할인) -> <i> 태그
                desc = dl.select_one("i").text.strip()
                
                # "유의사항" 같은 불필요한 정보는 제외하고 싶다면 조건 추가
                if category != "유의사항":
                    benefits_list.append(f"[{category}] {desc}")
            except AttributeError:
                continue # 태그가 비어있거나 구조가 다르면 건너뜀
        
        # 리스트를 하나의 문자열로 합침 (예: "[공과금] 10% 할인, [편의점] 10% 할인")
        benefits_str = ", ".join(benefits_list)


        # 4. 데이터 저장
        # ---------------------------------------------------------
        final_data.append({
            "카드명": card_name,
            "카드사": corp_name,
            "연회비": annual_fee,
            "전월실적": performance,
            "주요혜택": benefits_str,
            "상세페이지": link
        })
        
    except Exception as e:
        print(f"Error processing {link}: {e}")
        continue

# 크롤링 종료 후 브라우저 닫기
driver.quit()

# 저장할 경로와 파일명 지정
# r"..." 을 붙이면 윈도우 경로의 백슬래시(\) 오류를 방지할 수 있습니다.
save_path = r"C:\Users\muddy\Downloads\card_gorilla_results.xlsx"

# 데이터프레임 생성
df = pd.DataFrame(final_data)

try:
    # 엑셀로 저장 (engine='openpyxl' 명시)
    df.to_excel(save_path, index=False, engine='openpyxl')
    print(f"저장 성공! 파일 위치: {save_path}")
    print(f"총 {len(df)}개의 카드 정보가 저장되었습니다.")
except Exception as e:
    print(f"저장 중 에러 발생: {e}")

총 100개의 카드 링크를 수집했습니다.
수집된 링크 예시: https://www.card-gorilla.com/card/detail/13
상세 페이지 크롤링을 시작합니다...
[1/100] 이동 중: https://www.card-gorilla.com/card/detail/13
[2/100] 이동 중: https://www.card-gorilla.com/card/detail/2885
[3/100] 이동 중: https://www.card-gorilla.com/card/detail/2687
[4/100] 이동 중: https://www.card-gorilla.com/card/detail/49
[5/100] 이동 중: https://www.card-gorilla.com/card/detail/1909
[6/100] 이동 중: https://www.card-gorilla.com/card/detail/51
[7/100] 이동 중: https://www.card-gorilla.com/card/detail/2441
[8/100] 이동 중: https://www.card-gorilla.com/card/detail/2663
[9/100] 이동 중: https://www.card-gorilla.com/card/detail/2835
[10/100] 이동 중: https://www.card-gorilla.com/card/detail/2330
[11/100] 이동 중: https://www.card-gorilla.com/card/detail/2376
[12/100] 이동 중: https://www.card-gorilla.com/card/detail/2759
[13/100] 이동 중: https://www.card-gorilla.com/card/detail/2886
[14/100] 이동 중: https://www.card-gorilla.com/card/detail/2609
[15/100] 이동 중: https://www.card-gorilla.com/card/detail/666
[1

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# =================================================
# [자동화된 날짜 리스트 생성]
# =================================================
date_range = pd.date_range(start="2021-01-01", end="2026-02-01", freq="MS")
target_dates = [date.strftime("%Y-%m-%d") for date in date_range][::-1]
print(f"총 {len(target_dates)}개월치 데이터를 크롤링합니다.")

# 브라우저 설정
options = webdriver.ChromeOptions()
# options.add_argument('headless') 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_data = []

try:
    for date in target_dates:
        print(f"▶ {date} 랭킹 수집 시작...")
        
        url = f"https://www.card-gorilla.com/chart/top100?term=monthly&date={date}"
        driver.get(url)
        time.sleep(3)
        
        # 스크롤 내리기
        for i in range(1, 5):
            driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight * {i/4});")
            time.sleep(1)
        
        # =================================================
        # [PART 1] 1위 카드 정보 추출
        # =================================================
        try:
            winner_rank = driver.find_element(By.CSS_SELECTOR, "div.winner").text.strip()
            winner_container = driver.find_element(By.CSS_SELECTOR, "span.card_name")
            winner_corp = winner_container.find_element(By.CSS_SELECTOR, "p.corp_name").text.strip()
            
            winner_name_full = winner_container.text
            winner_name = winner_name_full.replace(winner_corp, "").strip()
            
            if winner_name: # 이름이 있는 경우에만 저장
                all_data.append({
                    "기준년월": date,
                    "순위": winner_rank,
                    "카드명": winner_name,
                    "카드사": winner_corp
                })
                print(f"   - [1위] {winner_name} 완료")
            
        except Exception as e:
            print(f"   - 1위 데이터 수집 실패: {e}")

        # =================================================
        # [PART 2] 2위 ~ 100위 (빈 행 필터링 추가)
        # =================================================
        try:
            ranks = driver.find_elements(By.CSS_SELECTOR, "div.num")
            areas = driver.find_elements(By.CSS_SELECTOR, "div.name_area")
            
            count = 0
            for rank_el, area_el in zip(ranks, areas):
                r_text = rank_el.text.strip()
                
                try:
                    # 텍스트 추출 시도
                    n_text = area_el.find_element(By.CSS_SELECTOR, "p.card_name").text.strip()
                    c_text = area_el.find_element(By.CSS_SELECTOR, "p.corp_name").text.strip()
                    
                    # [핵심 수정] 순위와 카드명이 모두 존재할 때만 리스트에 추가
                    # r_text가 비어있거나(""), 카드명이 없으면 저장하지 않음
                    if r_text and n_text and r_text != "1":
                        all_data.append({
                            "기준년월": date,
                            "순위": r_text,
                            "카드명": n_text,
                            "카드사": c_text
                        })
                        count += 1
                        
                except Exception:
                    # p.card_name 등을 찾지 못한 경우(빈 껍데기 태그 등)는 조용히 넘어감
                    continue
            
            print(f"   - [2위~] {count}개 데이터 정상 수집 완료")
            
        except Exception as e:
            print(f"   - 리스트 수집 중 에러: {e}")
            
        print("-" * 30)

except Exception as e:
    print(f"전체 프로세스 에러: {e}")

finally:
    driver.quit()

# 엑셀 저장
if all_data:
    df = pd.DataFrame(all_data)
    save_path = r"C:\Users\muddy\Downloads\card_gorilla_monthly_top100.xlsx"
    df.to_excel(save_path, index=False, engine='openpyxl')
    print(f"\n[완료] 총 {len(df)}개 데이터 저장됨: {save_path}")
    print(df.head())
else:
    print("수집된 데이터가 없습니다.")